In [2]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [4]:
mode = 'local'
t = 'test'

In [26]:
sk_sub = pd.read_csv('resources/dataset/originals/speeds_evaluation__only_datetimes__2019.csv')

In [100]:
sub = pd.read_csv('~/Desktop/submission_last.csv')
sub.shape, sk_sub.shape

((1615, 5), (1615, 4))

In [101]:
sk_sub.head()

,KEY,KM,DATETIME_UTC,PREDICTION_STEP
0,344,538,1/4/2019 9:15,3
1,533,231,1/5/2019 9:30,0
2,204,751,1/4/2019 9:30,1
3,86,422,1/12/2019 8:15,3
4,188,494,1/12/2019 8:30,2


In [102]:
sub.head()

,KEY,KM,DATETIME_UTC,PREDICTION_STEP,SPEED_AVG
0,0,333,2019-01-07 08:00:00,0,233.301951
1,0,333,2019-01-07 08:15:00,1,234.867390
2,0,333,2019-01-07 08:30:00,2,232.170262
3,0,333,2019-01-07 08:45:00,3,233.858989
4,0,337,2019-01-07 08:00:00,0,213.213819


### check speeds before

In [103]:
sub['DATETIME_UTC'] = pd.to_datetime(sub['DATETIME_UTC'])

In [104]:
for i in range(4,0,-1):
    sub[f'd_{i}'] = sub['DATETIME_UTC'] - pd.Timedelta(15*i, unit='m')

In [105]:
sub.head()

,KEY,KM,DATETIME_UTC,PREDICTION_STEP,SPEED_AVG,d_4,d_3,d_2,d_1
0,0,333,2019-01-07 08:00:00,0,233.301951,2019-01-07 07:00:00,2019-01-07 07:15:00,2019-01-07 07:30:00,2019-01-07 07:45:00
1,0,333,2019-01-07 08:15:00,1,234.867390,2019-01-07 07:15:00,2019-01-07 07:30:00,2019-01-07 07:45:00,2019-01-07 08:00:00
2,0,333,2019-01-07 08:30:00,2,232.170262,2019-01-07 07:30:00,2019-01-07 07:45:00,2019-01-07 08:00:00,2019-01-07 08:15:00
3,0,333,2019-01-07 08:45:00,3,233.858989,2019-01-07 07:45:00,2019-01-07 08:00:00,2019-01-07 08:15:00,2019-01-07 08:30:00
4,0,337,2019-01-07 08:00:00,0,213.213819,2019-01-07 07:00:00,2019-01-07 07:15:00,2019-01-07 07:30:00,2019-01-07 07:45:00


In [106]:
speeds = data.speeds_original('test2')

In [107]:
#merge speeds
sub = sub.rename(columns={'DATETIME_UTC':'pDATETIME_UTC'})
for i in range(4,0,-1):
    sub = sub.merge(speeds[['KEY','KM','DATETIME_UTC','SPEED_AVG']].rename(columns={'SPEED_AVG':f'SPEED_AVG_{i}'}), \
              left_on=['KEY','KM',f'd_{i}'], right_on=['KEY','KM','DATETIME_UTC'], how='left')
    sub.drop(f'd_{i}', axis=1, inplace=True)

sub.drop([col for col in sub.columns if col.startswith('DATETIME_UTC')], axis=1, inplace=True)

In [112]:
sub.head(50)

,KEY,KM,pDATETIME_UTC,PREDICTION_STEP,SPEED_AVG,SPEED_AVG_4,SPEED_AVG_3,SPEED_AVG_2,SPEED_AVG_1
0,0,333,2019-01-07 08:00:00,0,233.301951,237.058631,231.673888,230.030642,235.444683
1,0,333,2019-01-07 08:15:00,1,234.867390,231.673888,230.030642,235.444683,NaN
2,0,333,2019-01-07 08:30:00,2,232.170262,230.030642,235.444683,NaN,NaN
3,0,333,2019-01-07 08:45:00,3,233.858989,235.444683,NaN,NaN,NaN
4,0,337,2019-01-07 08:00:00,0,213.213819,210.505779,209.727301,214.520703,210.075327
5,0,337,2019-01-07 08:15:00,1,214.108209,209.727301,214.520703,210.075327,NaN
6,0,337,2019-01-07 08:30:00,2,214.349301,214.520703,210.075327,NaN,NaN
7,0,337,2019-01-07 08:45:00,3,214.039185,210.075327,NaN,NaN,NaN
8,0,343,2019-01-01 13:30:00,0,218.630455,224.763314,213.238330,219.865793,218.298455
9,0,343,2019-01-01 13:45:00,1,218.856673,213.238330,219.865793,218.298455,NaN


In [99]:
sub.shape

(1615, 13)